In [80]:
from datetime import datetime
import pandas as pd
import pandas_datareader.data as web

def getStock(Stock):
    stock_df = web.DataReader(Stock, 'yahoo', start_time, now_time)
    return stock_df
if __name__ == '__main__':
    now_time = datetime.now()
    start_time = datetime(now_time.year - 8, now_time.month, now_time.day)
    stock_df = getStock('TSLA')


In [81]:
stock_df.dropna()

,High,Low,Open,Close,Volume,Adj Close
Date,,,,,,
2014-05-23,41.551998,40.500000,40.905998,41.459999,20034000.0,41.459999
2014-05-27,42.773998,41.540001,41.703999,42.312000,26705500.0,42.312000
2014-05-28,42.554001,41.051998,42.004002,42.048000,27475500.0,42.048000
2014-05-29,42.498001,41.543999,42.113998,42.048000,18462500.0,42.048000
2014-05-30,42.959999,41.403999,42.060001,41.554001,27905500.0,41.554001
...,...,...,...,...,...,...
2022-05-17,764.479980,728.849976,747.359985,761.609985,26745400.0,761.609985
2022-05-18,760.500000,700.809998,744.520020,709.809998,29270600.0,709.809998
2022-05-19,734.000000,694.109985,707.000000,709.419983,30098900.0,709.419983


In [82]:
stock_df.head()

,High,Low,Open,Close,Volume,Adj Close
Date,,,,,,
2014-05-23,41.551998,40.500000,40.905998,41.459999,20034000.0,41.459999
2014-05-27,42.773998,41.540001,41.703999,42.312000,26705500.0,42.312000
2014-05-28,42.554001,41.051998,42.004002,42.048000,27475500.0,42.048000
2014-05-29,42.498001,41.543999,42.113998,42.048000,18462500.0,42.048000
2014-05-30,42.959999,41.403999,42.060001,41.554001,27905500.0,41.554001


In [83]:
import numpy as np
from scipy.stats import norm
import scipy.optimize as opt
import pandas as pd
import datetime
import time
from arch import arch_model
import matplotlib.pyplot as plt
from numba import jit
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import mean_absolute_error as mae
import warnings
import plotly.express as px
import plotly.graph_objects as go
import math
from datetime import timedelta
warnings.filterwarnings('ignore')

In [84]:
# stock_df.reset_index(inplace=True)
stock_df.sort_values(by=['Date'],ascending=False)

,High,Low,Open,Close,Volume,Adj Close
Date,,,,,,
2022-05-23,679.955078,638.059998,655.020020,674.900024,28945309.0,674.900024
2022-05-20,721.580017,633.000000,713.989990,663.900024,48190600.0,663.900024
2022-05-19,734.000000,694.109985,707.000000,709.419983,30098900.0,709.419983
2022-05-18,760.500000,700.809998,744.520020,709.809998,29270600.0,709.809998
2022-05-17,764.479980,728.849976,747.359985,761.609985,26745400.0,761.609985
...,...,...,...,...,...,...
2014-05-30,42.959999,41.403999,42.060001,41.554001,27905500.0,41.554001
2014-05-29,42.498001,41.543999,42.113998,42.048000,18462500.0,42.048000
2014-05-28,42.554001,41.051998,42.004002,42.048000,27475500.0,42.048000


In [85]:
ret = 100 * (stock_df.pct_change()[1:]['Adj Close'])
realized_vol = ret.rolling(5).std()

In [86]:
realized_vol

Date
2014-05-27         NaN
2014-05-28         NaN
2014-05-29         NaN
2014-05-30         NaN
2014-06-02    1.403346
                ...   
2022-05-17    6.305702
2022-05-18    5.897573
2022-05-19    5.898081
2022-05-20    5.223651
2022-05-23    5.201530
Name: Adj Close, Length: 2014, dtype: float64

In [87]:
newnames = {'Adj Close':'Realized Volatility'} # For Visualizations
fig = px.line(realized_vol, template='plotly_white', labels={'value':'Volatility'}, title='Realized Volatility')
fig.for_each_trace(lambda t: t.update(name = newnames[t.name],
                                      legendgroup = newnames[t.name],
                                      hovertemplate = t.hovertemplate.replace(t.name, newnames[t.name])
                                     )
                  )
fig.update_layout(
    title_x=0.5,
    legend_title="Legend"
)
fig.show()

In [88]:
period_volatility = {}
daily_volatility = ret.std()
print('Daily volatility: ', '{:.2f}%'.format(daily_volatility))
period_volatility['Daily Volatility'] = round(daily_volatility, 2)

monthly_volatility = math.sqrt(21) * daily_volatility
print ('Monthly volatility: ', '{:.2f}%'.format(monthly_volatility))
period_volatility['Monthly Volatility'] = round(monthly_volatility, 2)

annual_volatility = math.sqrt(252) * daily_volatility
print ('Annual volatility: ', '{:.2f}%'.format(annual_volatility ))
period_volatility['Annual Volatility'] = round(annual_volatility, 2)

Daily volatility:  3.46%
Monthly volatility:  15.88%
Annual volatility:  55.00%


In [89]:
n = 300
split_date = ret.iloc[-n:].index

In [90]:
retv = ret.values

fig = px.line(x=stock_df.index[1:], y=ret, template='plotly_white', labels={'x':'Date', 'y':'Daily Returns'}, 
              title='Volatility clustering')
fig.update_layout(
    title_x=0.5
)
fig.show()

# SVR-GARCH

In [92]:
from sklearn.svm import SVR
from scipy.stats import uniform as sp_rand
from sklearn.model_selection import RandomizedSearchCV

realized_vol = ret.rolling(5).std() 
realized_vol = pd.DataFrame(realized_vol)
realized_vol.reset_index(drop=True, inplace=True)

In [94]:
returns_svm = ret ** 2
returns_svm = returns_svm.reset_index()
del returns_svm['Date']

In [96]:
X = pd.concat([realized_vol, returns_svm], axis=1, ignore_index=True)
X = X[4:].copy()
X = X.reset_index()
X.drop('index', axis=1, inplace=True)

In [97]:
X

,0,1
0,1.403346,2.183304
1,0.701887,0.013747
2,0.705991,0.214876
3,1.154648,2.035023
4,1.097081,0.376772
...,...,...
2005,6.305702,26.430064
2006,5.897573,46.258767
2007,5.898081,0.003019
2008,5.223651,41.171518


In [99]:
realized_vol = realized_vol.dropna().reset_index()
realized_vol.drop('index', axis=1, inplace=True)

In [100]:
svr_lin = SVR(kernel='linear') 
# svr_rbf = SVR(kernel='rbf')

In [108]:
para_grid = {'gamma': sp_rand(), 'C': sp_rand(), 'epsilon': sp_rand()}
clf = RandomizedSearchCV(svr_lin, para_grid)
clf.fit(X.iloc[:-n].values, realized_vol.iloc[1:-(n-1)].values.reshape(-1,))
predict_svr_lin = clf.predict(X.iloc[-n:])

predict_svr_lin = pd.DataFrame(predict_svr_lin)
predict_svr_lin.index = ret.iloc[-n:].index

rmse_svr = np.sqrt(mse(realized_vol.iloc[-n:] / 100, predict_svr_lin / 100))
print('The RMSE value of SVR with Linear Kernel is {:.6f}'.format(rmse_svr))
mse_svr = mse(realized_vol.iloc[-n:] / 100, predict_svr_lin / 100)
print('The MSE value of SVR with Linear Kernel is {:.7f}'.format(mse_svr))

mae_svr = mae(realized_vol.iloc[-n:] / 100, predict_svr_lin / 100)
print('The MAE value of SVR with Linear Kernel is {:.6f}'.format(mae_svr))


The RMSE value of SVR with Linear Kernel is 0.002282
The MSE value of SVR with Linear Kernel is 0.0000052
The MAE value of SVR with Linear Kernel is 0.001724


In [109]:
realized_vol.index = ret.iloc[4:].index

svr_lin_forecast_df = predict_svr_lin / 100
svr_lin_forecast_df.columns = ['Preds']
fig = px.line(realized_vol / 100, template='plotly_white', labels={'value':'Volatility'}, title='Volatility Prediction with SVR-GARCH (Linear)')
fig.for_each_trace(lambda t: t.update(name = newnames[t.name],
                                      legendgroup = newnames[t.name],
                                      hovertemplate = t.hovertemplate.replace(t.name, newnames[t.name])
                                     )
                  )
fig.add_trace(go.Scatter(x=svr_lin_forecast_df.index, y=svr_lin_forecast_df['Preds'], mode='lines', name='SVR-GARCH Predictions'))
fig.update_layout(
    title_x=0.5,
    legend_title="Legend"
)
fig.show()

In [110]:
from datetime import date
dt = date.today()
svr_lin_forecast_df.loc[[dt]].values[0].item()

0.047800915186727376

# Neural Networks

In [13]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [14]:
model = keras.Sequential(
    [layers.Dense(256, activation="relu"),
    layers.Dense(128, activation="relu"),
    layers.Dense(1, activation="linear"),])

In [15]:
model.compile(loss='mse', optimizer='rmsprop')

In [16]:
epochs_trial = np.arange(100, 400, 4)
batch_trial = np.arange(100, 400, 4)

In [44]:

DL_pred = []
DL_RMSE = []
DL_MSE = []
DL_MAE = []
for i, j, k in zip(range(4), epochs_trial, batch_trial):
    model.fit(X.iloc[:-n].values, realized_vol.iloc[1:-(n-1)].values.reshape(-1,),
              batch_size=k, epochs=j, verbose=False)
    DL_predict = model.predict(np.asarray(X.iloc[-n:]))
    DL_RMSE.append(np.sqrt(mse(realized_vol.iloc[-n:] / 100, DL_predict.flatten() / 100)))
    DL_MSE.append(mse(realized_vol.iloc[-n:] / 100, DL_predict.flatten() / 100))
    DL_MAE.append(mae(realized_vol.iloc[-n:] / 100, DL_predict.flatten() / 100))
    DL_pred.append(DL_predict)
    print('DL_RMSE_{}:{:.6f}'.format(i+1, DL_RMSE[i]))     
    print('DL_MSE_{}:{:.6f}'.format(i+1, DL_MSE[i]))    
    print('DL_MAE_{}:{:.6f}'.format(i+1, DL_MAE[i]))    


10/10 [==============================] - 0s 912us/step
DL_RMSE_1:0.002642
DL_MSE_1:0.000007
DL_MAE_1:0.002264
10/10 [==============================] - 0s 2ms/step
DL_RMSE_2:0.001430
DL_MSE_2:0.000002
DL_MAE_2:0.001078
10/10 [==============================] - 0s 2ms/step
DL_RMSE_3:0.001310
DL_MSE_3:0.000002
DL_MAE_3:0.000962
10/10 [==============================] - 0s 2ms/step
DL_RMSE_4:0.001300
DL_MSE_4:0.000002
DL_MAE_4:0.000942


In [46]:
DL_predict = pd.DataFrame(DL_pred[DL_RMSE.index(min(DL_RMSE))])
DL_predict.index = ret.iloc[-n:].index

In [47]:
nn_forecast_df = DL_predict / 100
nn_forecast_df.columns = ['Preds']
fig = px.line(realized_vol / 100, template='plotly_white', labels={'value':'Volatility'}, title='Volatility Prediction with Neural Networks')
fig.for_each_trace(lambda t: t.update(name = newnames[t.name],
                                      legendgroup = newnames[t.name],
                                      hovertemplate = t.hovertemplate.replace(t.name, newnames[t.name])
                                     )
                  )
fig.add_trace(go.Scatter(x=nn_forecast_df.index, y=nn_forecast_df['Preds'], mode='lines', name='Neural Network Predictions'))
fig.update_layout(
    title_x=0.5,
    legend_title="Legend"
)
fig.show()

In [78]:
from datetime import date
dt = date.today()
nn_forecast_df.loc[[dt]].values[0].item()

0.033996690064668655